<a href="https://colab.research.google.com/github/Kuyas/TimeSeriesEntropy/blob/master/Pollution_Data_Entropy_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import math
import matplotlib.pyplot as plt
from tabulate import tabulate
import pandas as pd
import time as time
import seaborn as sns
import pickle
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(11, 4)})

In [0]:
def save_obj(obj, name ):
    with open('/content/drive/My Drive/TimeSeries/run4/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
    print("saved: ",name)
def load_obj(name ):
    with open('/content/drive/My Drive/TimeSeries/run4/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

## Questions

1.   Why Mse is so spiky? And what do the spikes mean?
2. Do we see same pattern/trend in mse and cmse for a location?
3. Do different pollutants behave differently and if so, why?
4. What is the effect of different parameters? How do plots change between different parameter values.
5. Are the patterns signature of a particular location? Can we classify locations based on these?

## Tasks
1. Make experiments more focussed - pick locations, pollutants, time scales etc. Generate general stats (mean, std, lengths etc.) for the time series at this stage.
2. Try different values of m for chosen pollutants and locations. (Make the code/plot work for one observation point and share with me. If it looks ok, we run experiments for other points.)
3. Try different values of r for chosen pollutants and locations
4. You can do 3-d plots as done in the sapien labs link I shared with you.
5. Analyse the plots and record the key observations.




## Task 1 - 
- Pick Locations - [Aotizhongxin, Changping, Huairou, Shunyi, Wanliu]
- Pollutants - PM2.5 CO
- Generate general stats - mean, std, length

In [0]:
df = pd.read_csv("/content/drive/My Drive/TimeSeries/PRSA_Data_Aotizhongxin_20130301-20170228.csv") #read csv
df1 = pd.to_datetime(df.iloc[:,1:5]) # convert the year-month-day-hour column to datetime format for time series
Aotizhongxin = pd.concat([df, df1], axis=1, sort=False) # concat the datetime to original csv
Aotizhongxin = Aotizhongxin.rename(columns = {0:"DateTime"}) # rename the concat'd column
Aotizhongxin = Aotizhongxin.set_index('DateTime') # make the concat'd column as the index/serial
cols_to_norm = ['PM2.5','PM10','SO2','NO2','CO','O3','TEMP','PRES','DEWP','RAIN','WSPM']
# Aotizhongxin[cols_to_norm] = Aotizhongxin[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min())) # Normalize the columns

Aotizhongxin_PM25 = Aotizhongxin.dropna(subset= ['PM2.5'])
Aotizhongxin_CO = Aotizhongxin.dropna(subset= ['CO'])

Aotizhongxin_only_na_PM25 = Aotizhongxin[~Aotizhongxin.index.isin(Aotizhongxin_PM25.index)]
Aotizhongxin_only_na_CO = Aotizhongxin[~Aotizhongxin.index.isin(Aotizhongxin_CO.index)]

Aotizhongxin_PM25 = Aotizhongxin_PM25['PM2.5'] # The PM2.5 series with nan removed
Aotizhongxin_CO = Aotizhongxin_CO['CO'] # The CO series with nan removed

""" STATS """
print("Aotizhongxin_PM25")
print("mean: ",np.mean(Aotizhongxin_PM25))
print("std: ",np.std(Aotizhongxin_PM25))
print("length: ",len(Aotizhongxin_PM25))
print("max value: ",Aotizhongxin_PM25.max())
print("min value: ",Aotizhongxin_PM25.min())
print("coeff. of variation",np.std(Aotizhongxin_PM25)/np.mean(Aotizhongxin_PM25))

print("\n\nAotizhongxin_CO")
print("mean: ",np.mean(Aotizhongxin_CO))
print("std: ",np.std(Aotizhongxin_CO))
print("length: ",len(Aotizhongxin_CO))
print("max value: ",Aotizhongxin_CO.max())
print("min value: ",Aotizhongxin_CO.min())
print("coeff. of variation",np.std(Aotizhongxin_CO)/np.mean(Aotizhongxin_CO))

Aotizhongxin_PM25
mean:  82.77361082632768
std:  82.13449056896356
length:  34139
max value:  898.0
min value:  3.0
coeff. of variation 0.9922786954563925


Aotizhongxin_CO
mean:  1262.9451453977408
std:  1221.4178897636355
length:  33288
max value:  10000.0
min value:  100.0
coeff. of variation 0.9671187178750925


In [0]:
df = pd.read_csv("/content/drive/My Drive/TimeSeries/PRSA_Data_Changping_20130301-20170228.csv") #read csv
df1 = pd.to_datetime(df.iloc[:,1:5]) # convert the year-month-day-hour column to datetime format for time series
Changping = pd.concat([df, df1], axis=1, sort=False) # concat the datetime to original csv
Changping = Changping.rename(columns = {0:"DateTime"}) # rename the concat'd column
Changping = Changping.set_index('DateTime') # make the concat'd column as the index/serial
cols_to_norm = ['PM2.5','PM10','SO2','NO2','CO','O3','TEMP','PRES','DEWP','RAIN','WSPM']
# Changping[cols_to_norm] = Changping[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min())) # Normalize the columns

Changping_PM25 = Changping.dropna(subset= ['PM2.5'])
Changping_CO = Changping.dropna(subset= ['CO'])

Changping_only_na_PM25 = Changping[~Changping.index.isin(Changping_PM25.index)]
Changping_only_na_CO = Changping[~Changping.index.isin(Changping_CO.index)]

Changping_PM25 = Changping_PM25['PM2.5'] # The PM2.5 series with nan removed
Changping_CO = Changping_CO['CO'] # The CO series with nan removed

""" STATS """
print("Changping_PM25")
print("mean: ",np.mean(Changping_PM25))
print("std: ",np.std(Changping_PM25))
print("length: ",len(Changping_PM25))
print("max value: ",Changping_PM25.max())
print("min value: ",Changping_PM25.min())
print("coeff. of variation",np.std(Changping_PM25)/np.mean(Changping_PM25))

print("\n\n Changping_CO")
print("mean: ",np.mean(Changping_CO))
print("std: ",np.std(Changping_CO))
print("length: ",len(Changping_CO))
print("max value: ",Changping_CO.max())
print("min value: ",Changping_CO.min())
print("coeff. of variation",np.std(Changping_CO)/np.mean(Changping_CO))

Changping_PM25
mean:  71.09974336541265
std:  72.32587148148997
length:  34290
max value:  882.0
min value:  2.0
coeff. of variation 1.0172451834288023


 Changping_CO
mean:  1152.3013445428255
std:  1103.0398396052353
length:  33543
max value:  10000.0
min value:  100.0
coeff. of variation 0.9572494598128455


In [0]:
df = pd.read_csv("/content/drive/My Drive/TimeSeries/PRSA_Data_Huairou_20130301-20170228.csv") #read csv
df1 = pd.to_datetime(df.iloc[:,1:5]) # convert the year-month-day-hour column to datetime format for time series
Huairou = pd.concat([df, df1], axis=1, sort=False) # concat the datetime to original csv
Huairou = Huairou.rename(columns = {0:"DateTime"}) # rename the concat'd column
Huairou = Huairou.set_index('DateTime') # make the concat'd column as the index/serial
cols_to_norm = ['PM2.5','PM10','SO2','NO2','CO','O3','TEMP','PRES','DEWP','RAIN','WSPM']
# Huairou[cols_to_norm] = Huairou[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min())) # Normalize the columns
Huairou_PM25 = Huairou.dropna(subset= ['PM2.5'])
Huairou_CO = Huairou.dropna(subset= ['CO'])

Huairou_only_na_PM25 = Huairou[~Huairou.index.isin(Huairou_PM25.index)]
Huairou_only_na_CO = Huairou[~Huairou.index.isin(Huairou_CO.index)]

Huairou_PM25 = Huairou_PM25['PM2.5'] # The PM2.5 series with nan removed
Huairou_CO = Huairou_CO['CO'] # The CO series with nan removed

""" STATS """
print("Huairou_PM25")
print("mean: ",np.mean(Huairou_PM25))
print("std: ",np.std(Huairou_PM25))
print("length: ",len(Huairou_PM25))
print("max value: ",Huairou_PM25.max())
print("min value: ",Huairou_PM25.min())
print("coeff. of variation",np.std(Huairou_PM25)/np.mean(Huairou_PM25))

print("\n\n Huairou_CO")
print("mean: ",np.mean(Huairou_CO))
print("std: ",np.std(Huairou_CO))
print("length: ",len(Huairou_CO))
print("max value: ",Huairou_CO.max())
print("min value: ",Huairou_CO.min())
print("coeff. of variation",np.std(Huairou_CO)/np.mean(Huairou_CO))

Huairou_PM25
mean:  69.62636686112984
std:  71.2238722965922
length:  34111
max value:  762.0
min value:  2.0
coeff. of variation 1.0229439723409466


 Huairou_CO
mean:  1022.5545449140955
std:  898.7248839608367
length:  33642
max value:  10000.0
min value:  100.0
coeff. of variation 0.8789016570615686


In [0]:
df = pd.read_csv("/content/drive/My Drive/TimeSeries/PRSA_Data_Shunyi_20130301-20170228.csv") #read csv
df1 = pd.to_datetime(df.iloc[:,1:5]) # convert the year-month-day-hour column to datetime format for time series
Shunyi = pd.concat([df, df1], axis=1, sort=False) # concat the datetime to original csv
Shunyi = Shunyi.rename(columns = {0:"DateTime"}) # rename the concat'd column
Shunyi = Shunyi.set_index('DateTime') # make the concat'd column as the index/serial
cols_to_norm = ['PM2.5','PM10','SO2','NO2','CO','O3','TEMP','PRES','DEWP','RAIN','WSPM']
# Shunyi[cols_to_norm] = Shunyi[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min())) # Normalize the columns
Shunyi_PM25 = Shunyi.dropna(subset= ['PM2.5'])
Shunyi_CO = Shunyi.dropna(subset= ['CO'])

Shunyi_only_na_PM25 = Shunyi[~Shunyi.index.isin(Shunyi_PM25.index)]
Shunyi_only_na_CO = Shunyi[~Shunyi.index.isin(Shunyi_CO.index)]

Shunyi_PM25 = Shunyi_PM25['PM2.5'] # The PM2.5 series with nan removed
Shunyi_CO = Shunyi_CO['CO'] # The CO series with nan removed


""" STATS """
print("Shunyi_PM25")
print("mean: ",np.mean(Shunyi_PM25))
print("std: ",np.std(Shunyi_PM25))
print("length: ",len(Shunyi_PM25))
print("max value: ",Shunyi_PM25.max())
print("min value: ",Shunyi_PM25.min())
print("coeff. of variation",np.std(Shunyi_PM25)/np.mean(Shunyi_PM25))

print("\n\n Shunyi_CO")
print("mean: ",np.mean(Shunyi_CO))
print("std: ",np.std(Shunyi_CO))
print("length: ",len(Shunyi_CO))
print("max value: ",Shunyi_CO.max())
print("min value: ",Shunyi_CO.min())
print("coeff. of variation",np.std(Shunyi_CO)/np.mean(Shunyi_CO))

Shunyi_PM25
mean:  79.49160200286961
std:  81.23055007910956
length:  34151
max value:  941.0
min value:  2.0
coeff. of variation 1.0218758715691398


 Shunyi_CO
mean:  1187.0639785927142
std:  1156.3565198778363
length:  32886
max value:  10000.0
min value:  100.0
coeff. of variation 0.9741315891403914


In [0]:
df = pd.read_csv("/content/drive/My Drive/TimeSeries/PRSA_Data_Wanliu_20130301-20170228.csv") #read csv
df1 = pd.to_datetime(df.iloc[:,1:5]) # convert the year-month-day-hour column to datetime format for time series
Wanliu = pd.concat([df, df1], axis=1, sort=False) # concat the datetime to original csv
Wanliu = Wanliu.rename(columns = {0:"DateTime"}) # rename the concat'd column
Wanliu = Wanliu.set_index('DateTime') # make the concat'd column as the index/serial
cols_to_norm = ['PM2.5','PM10','SO2','NO2','CO','O3','TEMP','PRES','DEWP','RAIN','WSPM']
# Wanliu[cols_to_norm] = Wanliu[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min())) # Normalize the columns
Wanliu_PM25 = Wanliu.dropna(subset= ['PM2.5'])
Wanliu_CO = Wanliu.dropna(subset= ['CO'])

Wanliu_only_na_PM25 = Wanliu[~Wanliu.index.isin(Wanliu_PM25.index)]
Wanliu_only_na_CO = Wanliu[~Wanliu.index.isin(Wanliu_CO.index)]

Wanliu_PM25 = Wanliu_PM25['PM2.5'] # The PM2.5 series with nan removed
Wanliu_CO = Wanliu_CO['CO'] # The CO series with nan removed

""" STATS """
print("Wanliu_PM25")
print("mean: ",np.mean(Wanliu_PM25))
print("std: ",np.std(Wanliu_PM25))
print("length: ",len(Wanliu_PM25))
print("max value: ",Wanliu_PM25.max())
print("min value: ",Wanliu_PM25.min())
print("coeff. of variation",np.std(Wanliu_PM25)/np.mean(Wanliu_PM25))

print("\n\n Wanliu_CO")
print("mean: ",np.mean(Wanliu_CO))
print("std: ",np.std(Wanliu_CO))
print("length: ",len(Wanliu_CO))
print("max value: ",Wanliu_CO.max())
print("min value: ",Wanliu_CO.min())
print("coeff. of variation",np.std(Wanliu_PM25)/np.mean(Wanliu_PM25))

Wanliu_PM25
mean:  83.37471599100398
std:  81.90438735763169
length:  34682
max value:  957.0
min value:  2.0
coeff. of variation 0.9823648138899695


 Wanliu_CO
mean:  1319.3535125706724
std:  1268.095262286608
length:  33252
max value:  10000.0
min value:  100.0
coeff. of variation 0.9823648138899695


# Entropy Measures

In [0]:
def sample_entropy(time_series, sample_length, tolerance = None):
    #The code below follows the sample length convention of Ref [1] so:
    M = sample_length - 1;

    time_series = np.array(time_series)
    if tolerance is None:
        tolerance = 0.1*np.std(time_series)

    n = len(time_series)

    #Ntemp is a vector that holds the number of matches. N[k] holds matches templates of length k
    Ntemp = np.zeros(M + 2)
    #Templates of length 0 matches by definition:
    Ntemp[0] = n*(n - 1) / 2


    for i in range(n - M - 1):
        template = time_series[i:(i+M+1)];#We have 'M+1' elements in the template
        rem_time_series = time_series[i+1:]

        searchlist = np.arange(len(rem_time_series) - M, dtype=np.int32)
        for length in range(1, len(template)+1):
            hitlist = np.abs(rem_time_series[searchlist] - template[length-1]) < tolerance
            Ntemp[length] += np.sum(hitlist)
            searchlist = searchlist[hitlist] + 1

    sampen =  - np.log(Ntemp[1:] / Ntemp[:-1])
    return sampen

def util_granulate_time_series(time_series, scale):
    n = len(time_series)
    b = int(np.fix(n / scale))
    temp = np.reshape(time_series[0:b*scale], (b, scale))
    cts = np.mean(temp, axis = 1)
    return cts


def multiscale_entropy(time_series, sample_length, tolerance = None, maxscale = None):
    if tolerance is None:
        #we need to fix the tolerance at this level. If it remains 'None' it will be changed in call to sample_entropy()
        tolerance = 0.1*np.std(time_series)
    if maxscale is None:
        maxscale = len(time_series)

    mse = np.zeros(maxscale)

    for i in range(maxscale):
        temp = util_granulate_time_series(time_series, i+1)
        mse[i] = sample_entropy(temp, sample_length, tolerance)[-1]
    return mse


def composite_multiscale_entropy(time_series, sample_length, scale, tolerance=None):
    cmse = np.zeros((scale, m))

    for i in range(scale):
        for j in range(i):
            tmp = util_granulate_time_series(time_series[j:], i + 1)
            cmse[i] += sample_entropy(tmp, sample_length, tolerance) / (i + 1)
    return cmse


## Task 2
- Try different values of m for chosen pollutants and locations. (Make the code/plot work for one observation point and share with me. If it looks ok, we run experiments for other points.)

In [0]:
places = ['Aotizhongxin','Changping','Huairou','Shunyi','Wanliu']
pollutant = ['PM2.5', 'CO']
pollution_series = [Aotizhongxin_PM25,Aotizhongxin_CO,Changping_PM25,Changping_CO,
                     Huairou_PM25,Huairou_CO,Shunyi_PM25,Shunyi_CO,
                     Wanliu_PM25,Wanliu_CO]

In [0]:
m_set = [1,2,3,4,5,6,7,8,9,10]
# 8-0, 8-8
for q in m_set:
  for k in range(10):
    print(q,k)
    series = pollution_series[k].tolist()
    m = q
    scale = 20
    tolerance_per = 0.2
    r = tolerance_per*np.std(series)
    filename = str(places[k//2])+"_"+pollutant[k%2]+"_m="+str(m)+"_r%="+str(tolerance_per)


    mse = multiscale_entropy(series,m,r,scale)
    cmse_temp = composite_multiscale_entropy(series,m,scale,r)
    cmse = np.zeros(scale)
    for i in range (scale):
      cmse[i] = np.nanmean(cmse_temp[i])
    save_obj(mse,filename+"_mse")
    print(mse)
    save_obj(cmse,filename+"_cmse")
    print(cmse)

1 0
saved:  Aotizhongxin_PM2.5_m=1_r%=0.2_mse
[1.66982676 1.68227225 1.67270272 1.67849823 1.67448111 1.68123001
 1.68648351 1.67863253 1.6843822  1.67893347 1.68591835 1.68148711
 1.68407173 1.68237562 1.68013464 1.67375216 1.67173118 1.67496033
 1.6778024  1.67303237]
saved:  Aotizhongxin_PM2.5_m=1_r%=0.2_cmse
[0.         0.84113613 1.1147664  1.25978462 1.3399408  1.39995038
 1.44299238 1.47076421 1.49651501 1.51294363 1.52994683 1.53992419
 1.55173496 1.56114253 1.56597261 1.57341184 1.57635523 1.58084989
 1.58201326 1.58564792]
1 1
saved:  Aotizhongxin_CO_m=1_r%=0.2_mse
[1.47781185 1.56830241 1.4714996  1.5155342  1.46523158 1.49277274
 1.45940657 1.48267535 1.49503528 1.46789358 1.48085256 1.46194406
 1.46612969 1.44754137 1.45679695 1.44982945 1.45370259 1.45969534
 1.45581377 1.45537004]
saved:  Aotizhongxin_CO_m=1_r%=0.2_cmse
[0.         0.7841512  0.98095462 1.13602128 1.1732982  1.24551862
 1.2521408  1.2952926  1.32954129 1.32296826 1.3480241  1.33953539
 1.35883255 1.35017

In [0]:
print(cmse)

[0.         0.34317502 0.51893791 0.64919877 0.74901591 0.84345779
 0.93963285 1.0268209  1.1111878  1.19348629 1.24943099 1.30968261
 1.37872978 1.42350428 1.41186217 1.45835921 1.52056497 1.55252906
        inf        inf]
